### Голубинов Владислав Андреевич 18-АС (18-ИВТ-1)
5 Вариант, 1 Сеть

In [29]:
#!unzip -uq "/content/drive/My Drive/colab data/lab2cut/pet.zip" -d "/content/drive/My Drive/colab data/lab2cut/pet"

In [30]:
import sys
import os
import numpy as np
from PIL import ImageFile
import tensorflow as tf
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k

In [31]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
nb_classes = 2  # number of classes
img_width, img_height = 299, 299  # change based on the shape/structure of your images
batch_size = 64  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
transformation_ratio = .05  # how aggressive will be the data augmentation/transformation

train_data_dir = "/content/drive/My Drive/colab data/lab2cut/pet/pet/train"
validation_data_dir = "/content/drive/My Drive/colab data/lab2cut/pet/pet/test"

In [32]:
base_model = Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)

In [33]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(nb_classes, activation='softmax')(x)

In [34]:
model = Model(base_model.input, predictions)
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [35]:
for layer in base_model.layers:
    layer.trainable = False

In [36]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=transformation_ratio,
                                   shear_range=transformation_ratio,
                                   zoom_range=transformation_ratio,
                                   cval=transformation_ratio,
                                   horizontal_flip=True,
                                   vertical_flip=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

In [37]:
os.makedirs(os.path.join(os.path.abspath(train_data_dir), '../preview'), exist_ok=True)
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

Found 2598 images belonging to 2 classes.


In [38]:
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size=(img_width, img_height),
                                                              batch_size=batch_size,
                                                              class_mode='categorical')

Found 402 images belonging to 2 classes.


In [39]:
model.compile(optimizer='nadam',
              loss='categorical_crossentropy', 
              metrics=['accuracy', 'mae'])

In [40]:
callbacks_list = [
    EarlyStopping(monitor='val_acc', patience=5, verbose=0)
]

In [42]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
history = model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.n // batch_size,
                    epochs=2,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.n // batch_size,
                    verbose=1,
                    callbacks=callbacks_list)

Epoch 1/2
40/40 [==============================] - 1088s 27s/step - loss: 0.3082 - accuracy: 0.9171 - mae: 0.2432 - val_loss: 0.0933 - val_accuracy: 0.9896 - val_mae: 0.0829
Epoch 2/2
40/40 [==============================] - 1096s 27s/step - loss: 0.1394 - accuracy: 0.9586 - mae: 0.1072 - val_loss: 0.0571 - val_accuracy: 0.9896 - val_mae: 0.0500


In [44]:
model.save_weights("/content/drive/My Drive/Colab models/lab2/model_wieghts.h5")

In [45]:
model.save("/content/drive/My Drive/Colab models/lab2/model_keras.h5")